# 

# Differential gene expression with DSeq2

Analysis of the dataset GSE160306

![image](figures/GSE160306_paper.png)

Let us start by downloading the data from GEO

In [179]:
!wget -P dataDownload -nc https://ftp.ncbi.nlm.nih.gov/geo/series/GSE160nnn/GSE160306/suppl/GSE160306_human_retina_DR_totalRNA_counts.txt.gz
import GEOparse
gse = GEOparse.get_GEO("GSE160306",destdir="./dataDownload")

2858.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
File ‘dataDownload/GSE160306_human_retina_DR_totalRNA_counts.txt.gz’ already there; not retrieving.



22-May-2023 06:15:41 INFO GEOparse - File already exist: using local version.
22-May-2023 06:15:41 INFO GEOparse - Parsing ./dataDownload/GSE160306_family.soft.gz: 
22-May-2023 06:15:41 DEBUG GEOparse - DATABASE: GeoMiame
22-May-2023 06:15:41 DEBUG GEOparse - SERIES: GSE160306
22-May-2023 06:15:41 DEBUG GEOparse - PLATFORM: GPL20301
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871112
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871113
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871114
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871115
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871116
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871117
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871118
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871119
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871120
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871121
22-May-2023 06:15:41 DEBUG GEOparse - SAMPLE: GSM4871122
22-May-2023 06:15:41 DEBUG GEOparse - 

We are going to use the python version of DSeq2 to analyze the differentially expressed genes starting from the CPM file

In [180]:
import os
import pandas as pd
import pickle as pkl

SAVE = True  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be
    # saved
    OUTPUT_PATH = "./output"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

now, let us create a dataframe with the information from the file. See this link for an explanation of the different types of data for their analysis

In [181]:
counts_df = pd.read_csv('dataDownload/GSE160306_human_retina_DR_totalRNA_counts.txt.gz',index_col='ensemblID')

clinical_df = pd.read_csv('dataProcessed/Disease group Experiment.csv',delimiter=";",index_col='sampleID')


In [182]:
counts_df.head()

,sample_09,sample_16,sample_43,sample_65,sample_07,sample_64,sample_51,sample_19,sample_80,sample_62,...,sample_18,sample_54,sample_03,sample_46,sample_34,sample_49,sample_23,sample_68,sample_44,sample_47
ensemblID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,90,145,167,187,155,147,213,161,288,206,...,317,128,135,143,297,174,123,179,92,103
ENSG00000000005,0,1,2,0,2,0,2,2,1,0,...,4,0,3,2,3,1,1,1,1,3
ENSG00000000419,332,344,348,331,394,298,384,293,407,332,...,489,364,265,371,555,346,336,295,336,278
ENSG00000000457,728,920,843,739,885,659,718,612,963,692,...,868,828,395,553,725,673,712,683,675,655
ENSG00000000460,451,497,577,545,411,587,663,739,658,583,...,489,579,525,535,486,379,438,421,406,410


In [183]:
clinical_df.head()

,sampleNumber,donor,gender,age,BMI,duration_diabetes,duration_diabetes_min,duration_diabetes_min_unit,diabetes_medication,other_medication,...,sample_site,sample_group,post_mortem_interval,post_mortem_interval_unit,buffer,buffer_volume,buffer_volume_unit,ammount_cells,ammount_cells_unit,RIN
sampleID,,,,,,,,,,,,,,,,,,,,,
sample_01,1,16,female,80,"30,23126382",NaN,0.0,years,NaN,"ASA, levoxyl, verpamil, aldactone, nasonex nas...",...,Macula,Control_Macula,285,min,Water,"16,95",ul,118,ng/ul,"6,2"
sample_02,2,31,male,73,"30,72577422",NaN,0.0,years,NaN,"isoptin, imdur, prozac, trazodone, ASA",...,Macula,Control_Macula,354,min,Water,"21,51",ul,93,ng/ul,"6,3"
sample_03,3,41,female,73,"28,24265381",NaN,0.0,years,NaN,"levoxyl, atenolol, avapro, dilantin, DDAVP, om...",...,Macula,Control_Macula,320,min,Water,"16,26",ul,123,ng/ul,"4,7"
sample_04,4,25,male,81,"24,95857988",NaN,0.0,years,NaN,"atenolol, DECADRON (STEROIDS), paroxetine, ASA...",...,Macula,Control_Macula,331,min,Water,"19,23",ul,104,ng/ul,"7,4"
sample_05,5,19,female,82,"32,38300173",NaN,0.0,years,NaN,"Plavix, demadex, levothyroxine, folic acid, me...",...,Macula,Control_Macula,305,min,Water,"14,93",ul,134,ng/ul,"8,3"


select only a subset of the columns in the clinical dataset to simplfy analysis

In [184]:
clinical_df=clinical_df[['disease_group_detailed','sample_site']]
clinical_df.head()


,disease_group_detailed,sample_site
sampleID,,
sample_01,Control,Macula
sample_02,Control,Macula
sample_03,Control,Macula
sample_04,Control,Macula
sample_05,Control,Macula


# Preprocess the data

In [185]:
counts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58051 entries, ENSG00000000003 to ENSG00000283699
Data columns (total 79 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   sample_09  58051 non-null  int64
 1   sample_16  58051 non-null  int64
 2   sample_43  58051 non-null  int64
 3   sample_65  58051 non-null  int64
 4   sample_07  58051 non-null  int64
 5   sample_64  58051 non-null  int64
 6   sample_51  58051 non-null  int64
 7   sample_19  58051 non-null  int64
 8   sample_80  58051 non-null  int64
 9   sample_62  58051 non-null  int64
 10  sample_27  58051 non-null  int64
 11  sample_15  58051 non-null  int64
 12  sample_29  58051 non-null  int64
 13  sample_59  58051 non-null  int64
 14  sample_41  58051 non-null  int64
 15  sample_71  58051 non-null  int64
 16  sample_35  58051 non-null  int64
 17  sample_78  58051 non-null  int64
 18  sample_33  58051 non-null  int64
 19  sample_01  58051 non-null  int64
 20  sample_60  58051 non-null  int6

In [186]:
counts_df.describe()

,sample_09,sample_16,sample_43,sample_65,sample_07,sample_64,sample_51,sample_19,sample_80,sample_62,...,sample_18,sample_54,sample_03,sample_46,sample_34,sample_49,sample_23,sample_68,sample_44,sample_47
count,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000,...,58051.000000,58051.000000,58051.000000,58051.000000,5.805100e+04,58051.000000,58051.000000,58051.000000,58051.000000,58051.000000
mean,437.887633,426.359408,491.568035,428.457270,589.181151,346.514858,493.966323,463.812027,491.485539,441.295085,...,608.942757,452.078879,464.504246,422.892767,6.054177e+02,453.346023,401.076467,479.183735,399.148215,455.384834
std,4063.475111,3829.223118,4432.092649,3946.315288,5335.985925,3405.197782,4379.881877,4212.085521,4196.334177,4834.854842,...,5602.479327,4317.101141,4388.759832,4146.684806,1.293004e+04,4061.059076,3524.489335,4069.474430,3580.222133,4573.057235
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000e+00,2.000000,1.000000,2.000000,1.000000,2.000000
75%,89.000000,85.000000,88.000000,90.000000,114.000000,75.000000,98.000000,83.000000,92.000000,73.000000,...,103.000000,88.000000,88.000000,86.000000,8.800000e+01,83.000000,73.000000,97.000000,73.000000,89.000000
max,620808.000000,616100.000000,625135.000000,678040.000000,888387.000000,522897.000000,605405.000000,619341.000000,636914.000000,850177.000000,...,835630.000000,632122.000000,629099.000000,728282.000000,2.983141e+06,626220.000000,527820.000000,613121.000000,592872.000000,752713.000000


first, we will transpose the data, as DSeq2 needs it in the form "samples" times "genes". We will ensure that the datatypes of the count values are numeric, too

In [187]:

counts_df=counts_df.T
counts_df.head()

ensemblID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000283690,ENSG00000283691,ENSG00000283692,ENSG00000283693,ENSG00000283694,ENSG00000283695,ENSG00000283696,ENSG00000283697,ENSG00000283698,ENSG00000283699
sample_09,90,0,332,728,451,32,351,103,834,1536,...,0,0,1,0,0,2,26,0,1,0
sample_16,145,1,344,920,497,41,197,155,752,1702,...,0,0,1,0,0,3,19,0,0,0
sample_43,167,2,348,843,577,41,225,236,901,1421,...,0,0,4,0,0,3,27,0,4,0
sample_65,187,0,331,739,545,46,170,220,890,1220,...,0,0,3,0,0,1,19,0,3,0
sample_07,155,2,394,885,411,52,209,370,1202,1991,...,0,0,2,0,0,0,26,0,4,0


Now remove all columns (genes) that have a mean lower than a given value to speed up the analysis

In [188]:
mean_to_discard=10.
max_to_discard=40.
counts_df=counts_df.loc[:,counts_df.max() >max_to_discard]
#counts_df.describe()

# Start DSeq2 analysis

To perform differential expression analysis (DEA), PyDESeq2 requires two types of inputs:

   * A count matrix of shape 'number of samples' x 'number of genes', containing read counts (non-negative integers),
   * Clinical data (or "column" data) of shape 'number of samples' x 'number of variables', containing sample annotations that will be used to split the data in cohorts.

Both should be provided as `pandas dataframes

In [189]:
#!pip install pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

In [190]:
counts_df.head()

ensemblID,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000283498,ENSG00000283503,ENSG00000283529,ENSG00000283536,ENSG00000283635,ENSG00000283652,ENSG00000283662,ENSG00000283667,ENSG00000283674,ENSG00000283696
sample_09,90,332,728,451,32,351,103,834,1536,1093,...,10,102,33,481,2,13,36,21,172,26
sample_16,145,344,920,497,41,197,155,752,1702,900,...,19,80,35,626,6,8,58,23,121,19
sample_43,167,348,843,577,41,225,236,901,1421,1234,...,14,89,36,325,2,15,33,22,141,27
sample_65,187,331,739,545,46,170,220,890,1220,807,...,18,89,35,281,3,26,14,34,150,19
sample_07,155,394,885,411,52,209,370,1202,1991,1559,...,19,110,38,443,14,16,36,56,163,26


In [191]:
counts_df.columns

Index(['ENSG00000000003', 'ENSG00000000419', 'ENSG00000000457',
       'ENSG00000000460', 'ENSG00000000938', 'ENSG00000000971',
       'ENSG00000001036', 'ENSG00000001084', 'ENSG00000001167',
       'ENSG00000001460',
       ...
       'ENSG00000283498', 'ENSG00000283503', 'ENSG00000283529',
       'ENSG00000283536', 'ENSG00000283635', 'ENSG00000283652',
       'ENSG00000283662', 'ENSG00000283667', 'ENSG00000283674',
       'ENSG00000283696'],
      dtype='object', name='ensemblID', length=21214)

counts_df=counts_df[['ENSG00000000003', 'ENSG00000000419', 'ENSG00000000457',
       'ENSG00000000460', 'ENSG00000000938', 'ENSG00000000971',
       'ENSG00000001036', 'ENSG00000001084', 'ENSG00000001167',
       'ENSG00000001460']]
counts_df.head()


counts_df = load_example_data(
    modality="raw_counts",
    dataset="synthetic",
    debug=False,
)

clinical_df = load_example_data(
    modality="clinical",
    dataset="synthetic",
    debug=False,
)

In [192]:
counts_df=counts_df.sort_index()

In [193]:
clinical_df

,disease_group_detailed,sample_site
sampleID,,
sample_01,Control,Macula
sample_02,Control,Macula
sample_03,Control,Macula
sample_04,Control,Macula
sample_05,Control,Macula
...,...,...
sample_76,NPDR + DME,Periphery
sample_77,NPDR + DME,Periphery
sample_78,PDR + DME,Periphery


In [194]:
counts_df.index

Index(['sample_01', 'sample_02', 'sample_03', 'sample_04', 'sample_05',
       'sample_06', 'sample_07', 'sample_08', 'sample_09', 'sample_10',
       'sample_11', 'sample_12', 'sample_13', 'sample_14', 'sample_15',
       'sample_16', 'sample_17', 'sample_18', 'sample_19', 'sample_20',
       'sample_21', 'sample_22', 'sample_23', 'sample_24', 'sample_25',
       'sample_26', 'sample_27', 'sample_28', 'sample_29', 'sample_30',
       'sample_31', 'sample_32', 'sample_33', 'sample_34', 'sample_35',
       'sample_36', 'sample_37', 'sample_38', 'sample_39', 'sample_40',
       'sample_41', 'sample_42', 'sample_43', 'sample_44', 'sample_46',
       'sample_47', 'sample_48', 'sample_49', 'sample_50', 'sample_51',
       'sample_52', 'sample_53', 'sample_54', 'sample_55', 'sample_56',
       'sample_57', 'sample_58', 'sample_59', 'sample_60', 'sample_61',
       'sample_62', 'sample_63', 'sample_64', 'sample_65', 'sample_66',
       'sample_67', 'sample_68', 'sample_69', 'sample_70', 'samp

In [195]:
clinical_df.index

Index(['sample_01', 'sample_02', 'sample_03', 'sample_04', 'sample_05',
       'sample_06', 'sample_07', 'sample_08', 'sample_09', 'sample_10',
       'sample_11', 'sample_12', 'sample_13', 'sample_14', 'sample_15',
       'sample_16', 'sample_17', 'sample_18', 'sample_19', 'sample_20',
       'sample_21', 'sample_22', 'sample_23', 'sample_24', 'sample_25',
       'sample_26', 'sample_27', 'sample_28', 'sample_29', 'sample_30',
       'sample_31', 'sample_32', 'sample_33', 'sample_34', 'sample_35',
       'sample_36', 'sample_37', 'sample_38', 'sample_39', 'sample_40',
       'sample_41', 'sample_42', 'sample_43', 'sample_44', 'sample_46',
       'sample_47', 'sample_48', 'sample_49', 'sample_50', 'sample_51',
       'sample_52', 'sample_53', 'sample_54', 'sample_55', 'sample_56',
       'sample_57', 'sample_58', 'sample_59', 'sample_60', 'sample_61',
       'sample_62', 'sample_63', 'sample_64', 'sample_65', 'sample_66',
       'sample_67', 'sample_68', 'sample_69', 'sample_70', 'samp

## Read counts modeling

The :class:`DeseqDataSet <dds.DeseqDataSet>` class has two mandatory arguments, `counts_df` and `clinical_df`, as well as a set of optional keyword arguments, among which:

* `design_factor`: the name of the column of clinical to be used as a design variable
* `refit_cooks`: whether to refit cooks outliers – this is advised, in general.

In [196]:
dds = DeseqDataSet(
    counts=counts_df,
    clinical=clinical_df,
    design_factors="sample_site",  # compare samples based on the "condition"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8,
)
dds

AnnData object with n_obs × n_vars = 79 × 21214
    obs: 'disease_group_detailed', 'sample_site'
    obsm: 'design_matrix'

Compute normalization factors

In [197]:
dds.fit_size_factors()

dds.obsm["size_factors"]

Fitting size factors...
... done in 0.42 seconds.



array([0.89646028, 1.05517127, 0.99445804, 0.98949054, 1.26995537,
       1.20447588, 1.32027092, 0.92263662, 1.00335288, 0.86239246,
       0.91944387, 1.18807744, 0.76917817, 0.92909739, 0.78823673,
       0.98048408, 0.86399368, 1.24671836, 0.97820047, 0.8730617 ,
       0.75286244, 0.93539895, 0.90253227, 0.94599617, 1.05018249,
       0.94979775, 0.64292689, 0.74747957, 0.95676905, 0.91917703,
       0.99544816, 0.99757752, 0.98173313, 1.11958352, 1.14800264,
       0.91971672, 1.09692822, 1.07007052, 1.22338523, 0.82048684,
       1.10090204, 0.99330977, 1.08491525, 0.8648279 , 0.98117537,
       0.99661428, 0.9450861 , 1.00698154, 0.97899851, 1.09618736,
       0.8928868 , 1.25787181, 0.99317932, 0.98983079, 1.03908959,
       0.89685322, 0.97010284, 0.96773549, 1.13661741, 1.24568419,
       0.90509963, 1.04279707, 0.80341532, 0.98802332, 1.04648983,
       0.99799966, 1.0992843 , 1.66431381, 1.41349545, 0.98545479,
       0.98919789, 0.99440689, 1.02677682, 1.07690767, 1.20093

Fit genewise dispersions

In [198]:
dds.fit_genewise_dispersions()

dds.varm["genewise_dispersions"]

Fitting dispersions...
... done in 30.03 seconds.



array([0.11148778, 0.01157398, 0.03034521, ..., 0.06759624, 0.11829724,
       0.05953774])

Fit dispersion trend coefficients

In [199]:
dds.fit_dispersion_trend()
dds.uns["trend_coeffs"]
dds.varm["fitted_dispersions"]

Fitting dispersion trend curve...
... done in 22.00 seconds.



array([0.07073882, 0.05688768, 0.04867486, ..., 0.23952738, 0.07118895,
       0.31240293])

Dispersion priors

In [200]:

dds.fit_dispersion_prior()
print(
    f"logres_prior={dds.uns['_squared_logres']}, sigma_prior={dds.uns['prior_disp_var']}"
)


logres_prior=0.7237082937331533, sigma_prior=0.6973940225774794



### MAP Dispersions

The `fit_MAP_dispersions` method filters the genes for which dispersion
shrinkage is applied.
Indeed, for genes whose MLE dispersions are too high above the trend curve,
the original MLE value is kept.
The final values of the dispersions that are used for downstream analysis is
stored in `dds.dispersions`.

In [201]:

dds.fit_MAP_dispersions()
dds.varm["MAP_dispersions"]
dds.varm["dispersions"]

Fitting MAP dispersions...
... done in 35.54 seconds.



array([0.1096183 , 0.01264203, 0.03091876, ..., 0.07495389, 0.11607257,
       0.0703911 ])

### Fit log fold changes

Note that in the `DeseqDataSet` object, the log-fold changes are stored in natural log scale, but that the results dataframe output by the `summary` method of `DeseqStats` displays LFCs in log2 scale (see later on).


In [202]:
dds.fit_LFC()
dds.varm["LFC"]

Fitting LFCs...
... done in 17.61 seconds.



,intercept,sample_site_Periphery_vs_Macula
ensemblID,,
ENSG00000000003,5.020521,0.368978
ENSG00000000419,5.807325,0.049432
ENSG00000000457,6.435280,0.162077
ENSG00000000460,6.167144,0.192656
ENSG00000000938,3.385091,0.250468
...,...,...
ENSG00000283652,2.371360,0.079231
ENSG00000283662,3.767486,1.247008
ENSG00000283667,3.449806,-0.218735


### Calculate Cooks distances and refit

In [203]:
dds.calculate_cooks()
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(dds, f)

Refitting 175 outliers.

Fitting dispersions...
... done in 0.75 seconds.

Fitting MAP dispersions...
... done in 0.69 seconds.

Fitting LFCs...
... done in 0.55 seconds.




## Statistical analysis

Statistical analysis with the :class:`DeseqStats <ds.DeseqStats>` class. The `DeseqDataSet` class has a unique mandatory arguments, `dds`, which should
be a *fitted* `DeseqDataSet` object, as well as a set of optional keyword arguments, among which:

* `alpha`: the p-value and adjusted p-value significance threshold
* `cooks_filter`: whether to filter p-values based on cooks outliers
* `independent_filter`: whether to perform independent filtering to correct p-value trends.


In [206]:

stat_res = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=False)


KeyError: 'sample_site_vs_Macula'

### Wald tests

In [ ]:
stat_res.run_wald_test()
stat_res.p_values

### Cooks filtering

In [ ]:

if stat_res.cooks_filter:
    stat_res._cooks_filtering()
stat_res.p_values

### P-value adjustment

In [ ]:


if stat_res.independent_filter:
    stat_res._independent_filtering()
else:
    stat_res._p_value_adjustment()

stat_res.padj



### Building a results dataframe

This dataframe is stored in the `results_df` attribute of the `DeseqStats` class.


In [ ]:

stat_res.summary()


if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

### LFC Shrinkage

For visualization or post-processing purposes, it might be suitable to perform LFC shrinkage. This is implemented by the `lfc_shrink` method.


In [ ]:

stat_res.lfc_shrink(coeff="condition_B_vs_A")

if SAVE:
    with open(
        os.path.join(OUTPUT_PATH, "shrunk_stat_results_detailed_pipe.pkl"), "wb"
    ) as f:
        pkl.dump(stat_res, f)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eb62cd42-7a03-4ad2-9b9b-ccfb1448eb9e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>